In [11]:
import os
import pandas as pd

def _unique_output_path(base_path):
    if not os.path.exists(base_path):
        return base_path
    root, ext = os.path.splitext(base_path)
    i = 1
    while True:
        candidate = f"{root} ({i}){ext}"
        if not os.path.exists(candidate):
            return candidate
        i += 1


def split_excel_sheet(
    df,
    sheet_name,
    input_excel_path,
    filename_col=0,
    value_col=1,
    overwrite=False,
    biological_rep=None,
    start_well=1,
):
    """
    Process a single sheet and return:
      - formatted DataFrame
      - output path
      - next starting well number

    Well numbers:
      * Assigned in blocks by Treatment (dose+volume) and numeric well_id
      * Up to 4 distinct numeric well_ids per Well number
      * All rows belonging to those well_ids share that Well
      * Well labels are ascending across the sheet (1,2,3,...) with no restart.
    """

    out_rows = []
    for idx in range(len(df)):
        name = df.iloc[idx, filename_col]
        if pd.isna(name):
            continue

        name = str(name).strip()
        base = os.path.splitext(os.path.basename(name))[0]
        parts = base.split('_')

        # Expected pattern: dose_volume_wellid_image_...
        # e.g. 100ug_75uL_07_1_C_0.tiff
        dose    = parts[0] if len(parts) > 0 else ""
        volume  = parts[1] if len(parts) > 1 else ""
        well_id = parts[2] if len(parts) > 2 else ""
        image   = parts[3] if len(parts) > 3 else ""
        result_val = df.iloc[idx, value_col] if value_col < df.shape[1] else None

        treatment = f"{dose}_{volume}"

        # try to parse numeric well index; fall back to string if needed
        try:
            well_num = int(well_id)
        except ValueError:
            well_num = None

        out_rows.append({
            "Filename": name,
            "Treatment": treatment,
            "Dose": dose,
            "Volume": volume,
            "Well_ID": well_id,
            "Well_Num": well_num,
            "ImageRaw": image,
            "Biological_Rep": biological_rep,
            "Results": result_val,
            "Original_Order": idx,
        })

    out_df = pd.DataFrame(out_rows)

    # ensure original order
    out_df = out_df.sort_values("Original_Order").reset_index(drop=True)

    # ------------------------------------------------------------------
    # BUILD WELL GROUPS PER TREATMENT
    # ------------------------------------------------------------------
    well_labels = pd.Series(index=out_df.index, dtype="Int64")
    current_well_label = start_well

    # Process each Treatment independently, but keep *global* well counter
    for treatment, sub_idx in out_df.groupby("Treatment", sort=False).groups.items():
        sub = out_df.loc[sub_idx]

        # get unique numeric well numbers in order of appearance
        unique_wells = []
        for w in sub["Well_Num"]:
            if pd.isna(w):
                continue
            if w not in unique_wells:
                unique_wells.append(w)

        # walk through unique_wells in chunks of up to 4,
        # assign same Well label to all rows whose Well_Num is in that chunk
        pos = 0
        while pos < len(unique_wells):
            chunk = unique_wells[pos:pos+4]  # up to 4 numeric well positions
            mask = (out_df["Treatment"] == treatment) & (out_df["Well_Num"].isin(chunk))
            well_labels.loc[mask] = current_well_label

            current_well_label += 1
            pos += 4

    out_df["Well"] = well_labels

    # Image index within each (Treatment, Well_Num) group
    out_df["Image"] = (
        out_df
        .groupby(["Treatment", "Well_Num"], sort=False)
        .cumcount() + 1
    )

    # final column selection, preserving original order
    out_df = out_df.sort_values("Original_Order")
    out_df = out_df[["Filename", "Treatment", "Biological_Rep", "Well", "Image", "Results"]]

    # compute next starting well for a subsequent sheet (if you want to carry across sheets)
    next_start_well = int(out_df["Well"].max()) + 1 if len(out_df) > 0 else start_well

    # write CSV
    in_dir = os.path.dirname(os.path.abspath(input_excel_path))
    in_stem = os.path.splitext(os.path.basename(input_excel_path))[0]
    safe_sheet_name = str(sheet_name).replace("/", "_").replace("\\", "_")
    out_path = os.path.join(in_dir, f"{in_stem}_{safe_sheet_name}_RMixedFormated.csv")
    if not overwrite:
        out_path = _unique_output_path(out_path)

    out_df.to_csv(out_path, index=False)
    print(f"Saved: {out_path}")
    print(f"  Sheet: {sheet_name}")
    print(f"  Biological Replicate: {biological_rep}")
    print(f"  Total rows: {len(out_df)}")
    print(f"  Wells numbered {out_df['Well'].min()} to {out_df['Well'].max()} (ascending, blocks of up to 4 numeric wells)\n")

    return out_df, out_path, next_start_well


def split_excel_all_sheets(
    input_excel_path,
    filename_col=0,
    value_col=1,
    overwrite=False,
    biological_reps=None,
    auto_number_bio_reps=True,
):
    """
    Process all sheets in an Excel file with ascending well numbering.
    """
    excel_file = pd.ExcelFile(input_excel_path)
    sheet_names = excel_file.sheet_names

    print(f"\nFound {len(sheet_names)} sheet(s) in Excel file:")
    for i, name in enumerate(sheet_names, 1):
        print(f"  {i}. {name}")
    print()

    # handle biological_reps as in your previous version
    if biological_reps is None and auto_number_bio_reps:
        print("Biological replicate assignment options:")
        print("  1. Auto-number sheets sequentially (Sheet 1 = Bio Rep 1, Sheet 2 = Bio Rep 2, etc.)")
        print("  2. Use same biological replicate for all sheets")
        print("  3. Manually enter for each sheet")
        while True:
            choice = input("\nSelect option (1, 2, or 3): ").strip()
            if choice == "1":
                biological_reps = {sheet_names[i]: i+1 for i in range(len(sheet_names))}
                print(f"\n✓ Auto-numbering: Sheets 1-{len(sheet_names)} will use Bio Reps 1-{len(sheet_names)}\n")
                break
            elif choice == "2":
                while True:
                    try:
                        bio_rep = int(input("Enter biological replicate number for ALL sheets: "))
                        biological_reps = bio_rep
                        print(f"\n✓ All sheets will use Bio Rep {bio_rep}\n")
                        break
                    except ValueError:
                        print("Please enter a valid integer.")
                break
            elif choice == "3":
                biological_reps = None
                auto_number_bio_reps = False
                print("\n✓ You will be prompted for each sheet\n")
                break
            else:
                print("Invalid choice. Please enter 1, 2, or 3.")

    results = {}
    current_start_well = 1

    for sheet_name in sheet_names:
        print(f"Processing sheet: {sheet_name}")
        print("-" * 50)

        df = pd.read_excel(input_excel_path, sheet_name=sheet_name)

        if isinstance(biological_reps, dict):
            bio_rep = biological_reps.get(sheet_name, None)
        elif isinstance(biological_reps, int):
            bio_rep = biological_reps
        else:
            while True:
                try:
                    bio_rep = int(input(f"Enter Biological Replicate number for sheet '{sheet_name}': "))
                    break
                except ValueError:
                    print("Please enter a valid integer number.")

        out_df, out_path, next_start_well = split_excel_sheet(
            df=df,
            sheet_name=sheet_name,
            input_excel_path=input_excel_path,
            filename_col=filename_col,
            value_col=value_col,
            overwrite=overwrite,
            biological_rep=bio_rep,
            start_well=current_start_well,
        )

        results[sheet_name] = (out_df, out_path)
        current_start_well = next_start_well

    print(f"\n{'='*50}")
    print(f"SUMMARY: Processed {len(results)} sheet(s)")
    print(f"{'='*50}")
    for sheet_name, (df, path) in results.items():
        print(f"  {sheet_name}: {len(df)} rows, wells {df['Well'].min()}–{df['Well'].max()} -> {os.path.basename(path)}")


# ================================================================================
# CALL THE SOURCE EXCEL FILE
# ================================================================================
EXCEL_FILE_PATH = '/Users/allisonpickle/Desktop/Microglia Morphology Stats/Microglia_Morphology.xlsx'
if not os.path.exists(EXCEL_FILE_PATH):
    print(f"ERROR: File not found at {EXCEL_FILE_PATH}")
    print("Please check the file path.")
else:
    results = split_excel_all_sheets(EXCEL_FILE_PATH, biological_reps=1)


Found 3 sheet(s) in Excel file:
  1. Area
  2. Perimeter
  3. Circularity

Processing sheet: Area
--------------------------------------------------
Saved: /Users/allisonpickle/Desktop/Microglia Morphology Stats/Microglia_Morphology_Area_RMixedFormated.csv
  Sheet: Area
  Biological Replicate: 1
  Total rows: 329
  Wells numbered 1 to 23 (ascending, blocks of up to 4 numeric wells)

Processing sheet: Perimeter
--------------------------------------------------
Saved: /Users/allisonpickle/Desktop/Microglia Morphology Stats/Microglia_Morphology_Perimeter_RMixedFormated.csv
  Sheet: Perimeter
  Biological Replicate: 1
  Total rows: 329
  Wells numbered 24 to 46 (ascending, blocks of up to 4 numeric wells)

Processing sheet: Circularity
--------------------------------------------------
Saved: /Users/allisonpickle/Desktop/Microglia Morphology Stats/Microglia_Morphology_Circularity_RMixedFormated.csv
  Sheet: Circularity
  Biological Replicate: 1
  Total rows: 329
  Wells numbered 47 to 69